# 데이터 프레임 코드 (draft)

In [3]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
drive.mount('/content/drive')

Mounted at /gdrive
Mounted at /content/drive


In [4]:
# 파일 불러오기 
guide = '/content/drive/MyDrive/Data/환경부가이드라인_전처리완.csv'

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv(guide)
del df['Unnamed: 0']

In [7]:
df.head()

,품목,구분,배출방법
0,가격표,종이,종이류로 배출
1,가구류,대형,대형폐기물로 배출
2,가발,종량제봉투,종량제 봉투로 배출
3,가습기,가전제품,폐가전제품으로 배출
4,가위,"종량제봉투, 재질별분리","재질에 맞게 배출하되, 여러재질이 섞인 경우, 종량제 봉투로 배출"


In [8]:
# 품목 리스트 생성 
item_list = df['품목'].tolist()

In [9]:
item_list[:5]

['가격표', '가구류', '가발', '가습기', '가위']

In [10]:
question = '가구류는 어떻게 분리해?'

# 사용자가 요청한 내용이 있는 경우 

for i in range(len(df['품목'])):
  if df['품목'][i] in question:
    print(df['배출방법'][i]+'해주세요')

대형폐기물로 배출해주세요


In [11]:
# 사용자가 요청한 내용이 없는 경우 - txt 파일로 저장

msg = input('제품명을 입력해주세요: ')
if msg in item_list:                # 음성으로 인식을 못한 경우, 제품명 text로 입력 받기 
  df_new = df[df['품목']== msg]
  meth = df_new['배출방법']
  method = ' ,'.join(map(str, meth))
  print(method + '합니다')
else:
  aw = input('등록되지 않은 제품입니다. 등록하시겠습니까? ')
  if aw == '네':                                        # 제품 등록, 파일 업데이트(txt)
    f = open("update.txt", 'a')
    f.write(msg)
    f.close()
  else:
    print('서비스를 종료합니다.')

제품명을 입력해주세요: 가위
재질에 맞게 배출하되, 여러재질이 섞인 경우, 종량제 봉투로 배출합니다


In [12]:
# 사용자가 요청한 내용이 없는 경우 - 데이터프레임 업데이트 

msg = input('제품명을 입력해주세요: ')    # (1) 음성으로 인식을 못한 경우, 제품명 text로 입력 받기 
if msg in item_list:                
  df_new = df[df['품목']== msg]
  meth = df_new['배출방법']
  method = ' ,'.join(map(str, meth))
  print(method + '합니다')
else:                     # (2) 등록되지 않은 경우       
  aw = input('등록되지 않은 제품입니다. 등록하시겠습니까? ')
  if aw == '네': 
    new_data = {'품목': msg, 
                '구분': '',
                '배출방법':''}
    df = df.append(new_data, ignore_index=True)  # 제품 등록, 파일 업데이트 (txt)
  else:
    print('서비스를 종료합니다.')

제품명을 입력해주세요: 가위
재질에 맞게 배출하되, 여러재질이 섞인 경우, 종량제 봉투로 배출합니다


### 함수 작성

- 문제점

1. '일회용 면도기'와 같이 띄어쓰기로 되어있는 단어 식별 어려움
2. '가구류'와 같이 특정한 종목의 분야인 경우 식별 어려움

In [13]:
# 단어 토큰화 및 간단한 조사 제거

def token_word(query): # query = '가위는 어떻게 분리해?'
  post_word = ['은', '는', '랑', '이랑'] # 제거할 조사
  query_split = query.split(' ') # 띄어쓰기로 구분하여 분리 ['가위는', '어떻게', '분리해?']

  new_query = [] # 조사가 제거되고 토근화될 단어 리스트
  for qs in query_split: # qs = '가위는'
    new_ch = ''
    
    for char in qs: # char => '가' '위' '는'
      if char not in post_word:
        new_ch += char
    
    new_query.append(new_ch)
  
  return new_query

In [14]:
# 분리된 질문에서 원하는 재활용품의 위치를 찾는 함수

def get_item_index(new_query, item_list):
  flag = True
  idx = -1 # 초기값 -1로 설정함으로써 이후에 flag값으로 사용
  if type(new_query) == str:
    flag = False

  for i in range(len(item_list)): # item_list의 길이만큼 순회하면서 new_quert가 배열인지 str인지에 따라 다른 진행
    if flag: # list
      for word in new_query:
        if word == item_list[i]:
          idx = i
          break
    else: # str
      if new_query == item_list[i]:
        idx = i
        break

  return idx      

In [30]:
def how_to_recycle(idx, df):
  if idx > -1:
    return(f"{df['품목'][idx]}(은/는) {df['배출방법'][idx]}해주세요")
  else:
    msg = input('제대로 인식하지 못했습니다. 제품명을 직접 입력해주세요. : ')
    new_idx = get_item_index(msg, item_list)
    if new_idx > -1:
      return(f"{df['품목'][new_idx]}(은/는) {df['배출방법'][new_idx]}해주세요")
    else:
      aw = input('등록되지 않은 제품입니다. 등록하시겠습니까? (네, 아니오) : ')
      if aw == '네':                                        # 제품 등록, 파일 업데이트(txt)
        f = open("update.txt", 'a')
        f.write(msg)
        f.close()

        new_data = {'품목': msg, 
                    '구분': '',
                    '배출방법':''}
        df = df.append(new_data, ignore_index=True)  # 제품 등록, 파일 업데이트 (txt)
      return('서비스를 종료합니다.')

In [25]:
query = '면도기는 어떻게 분리해?'

In [26]:
new_query = token_word(query)
idx = get_item_index(new_query, item_list)
how_to_recycle(idx, df)

제대로 인식하지 못했습니다. 제품명을 직접 입력해주세요. : 면도기
등록되지 않은 제품입니다. 등록하시겠습니까? (네, 아니오) : 네


'서비스를 종료합니다.'

### 네이버 클라우드 적용
- 정답만 음성으로 출력? 예외 상황도 음성으로 출력?
- test.py파일을 import 해야함 

In [2]:
from test import print_voice, text_voice

In [31]:
query = print_voice('가위.m4a')

In [32]:
# 방법 호출 
new_query = token_word(query)
idx = get_item_index(new_query, item_list)
how_to = how_to_recycle(idx, df)
text_voice(how_to)

200
